In [1]:
import numpy as np
import random as rn
import findspark
import sys
findspark.init("C:\\Users\\Software\\spark-3.3.1-bin-hadoop3\\bin\\..")
#findspark.init("D:\\Users\\software\\spark-3.2.1-bin-hadoop3.2\\bin\\..")

In [2]:
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [3]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [4]:
V_max= 5.0
W= 1.0
c_1 = 0.8
c_2 = 0.2
objetivo =[50,50,50]
ind=0
filas_x=[0,0,0]

In [5]:
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 1000000
posiciones_=[]
mejor_posiciones_locales_=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales_])
mejor_pos_global_=[]
mejor_pos_global_arr= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness_arr=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
best_global_fitness = max
accum=sc.accumulator(best_global_fitness)
mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
best_local_fitness= sc.broadcast(best_local_fitness_arr)

In [6]:
def InitParticles(part,N,M,bgf,blf):
    #global best_global_fitness
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        posiciones_.append(posicion)
        posiciones= np.array(posiciones_)
        mejor_posiciones_locales_=posiciones_
        mejor_posiciones_locales=np.array(mejor_posiciones_locales_)
        velocidad=[rn.uniform(-100,100) for j in range(N)]
        velocidades_.append(velocidad)
        velocidades= np.array(velocidades_)
        fit = MSE(posiciones[j],objetivo)
        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("bgf-> ",bgf)
        if fit < bgf:
            bgf = fit
            accum= fit
            mejor_pos_global = mejor_posiciones_locales[j][:]
    part= np.array([posiciones,velocidades,mejor_posiciones_locales])
    return blf,bgf,mejor_pos_global,part 

In [7]:
def fitnessEval(part,mpg,blf,bgf, ind):
    filas=part[0:1,ind]
    fit = MSE(filas[0],objetivo)
    if fit< blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
        part[2:,ind]=filas
        if fit < bgf:
            bgf = fit
            mpg = filas[0]
            accum= fit
    return part,mpg,blf,bgf

In [8]:
#ind=sc.accumulator(0)
def fitnessEval_x(part,mpg,blf,bgf):
    global ind
    filas=part[0:1]
    fit = MSE(filas[0],objetivo)
    if fit < blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
      #  part[2:,ind]=filas
        if fit < bgf:
            bgf = fit
            mpg = filas[0]
            accum= fit
    if ind<10:
        ind+=1
    return part,mpg,blf,bgf

In [9]:
#def posEval(part,mpl,mpg,ind):
def posEval(part,mpg,ind,N):
    velocidades=part[1:2,ind][0]
    mpl=part[2:,ind][0]
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[0:1,ind][0][k]) + c_2*r_2*(mpg[k] - part[0:1,ind][0][k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[0:1,ind][0][k] = part[0:1,ind][0][k] + velocidades[k]
    #return part,velocidades
    return part

In [10]:
best_local_fitness,best_global_fitness,mejor_pos_global,particulas=InitParticles(particulas,n,m,best_global_fitness,best_local_fitness)
#print("particulas[0:1]-> ",particulas[0:1][0])
#print("mejor velocidades-> ",particulas[1:2][0])
#print("mejor posiciones locales-> ",particulas[2:][0])
#print("mejor_pos_global-> ",mejor_pos_global)
#print("best_global_fitness",best_global_fitness)
#print("best_local_fitness",best_local_fitness)

In [11]:
rdd_master= sc.parallelize(particulas)
resultado = rdd_master.map(lambda x:fitnessEval_x(x,mejor_pos_global,best_local_fitness,best_global_fitness)).collect()
print("particulas-> ",particulas[0:1][0])
print("mejor_pos_global-> ",mejor_pos_global)

particulas->  [[ 8.89134221e+01  2.65268987e+00  5.30333516e+01]
 [ 8.37588128e+01  9.38483237e+01  2.57141091e+01]
 [ 7.52527751e+01 -8.05921443e+01 -5.70871287e+01]
 [ 2.58310661e+01 -3.02668293e+01 -5.28738600e+01]
 [ 6.24329315e+00  8.87617962e+01 -9.02369036e+01]
 [ 6.77800764e+01 -6.89426601e+01  7.89055292e+01]
 [-5.59252346e+01  7.37952368e+01 -8.50294586e+01]
 [-9.50494274e+01  7.96472737e+01  6.59987925e+01]
 [-1.18352714e+01 -6.91625940e+01  1.21673286e-02]
 [-1.33165297e+01  7.09729968e+01 -8.60582173e+01]]
mejor_pos_global->  [83.75881275 93.84832368 25.71410907]
